In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/cambrian_reco/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
import numpy as np
import pandas as pd
import json
import requests
sys.path.append('/Users/username/Desktop/ML/Recommendations/cambrian_reco/')

In [5]:
!pip install openai tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/f4/99/90fefb027bd962cd336f9f955a2f27782aac123ee7943cfc8a7bd3c1f198/tiktoken-0.5.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 4.8 MB/s eta 0:00:00a 0:00:01


In [6]:
import re
import pandas as pd
import numpy as np
import openai
import json
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

In [7]:
openai.api_key = ''

In [59]:
def get_prompt(topic):
    return f'''
    write upto 50 comma separated keywords which you would expect to find in financial articles related to the following topic {topic}. 
    25 keywords and 25 important nouns. If there are important companies or organisations - then definitely list them in the nouns. Use companies and organisations only if they are directly relevant for that topic. If they don't have high recall among users for that topic, then no need for those or any organisations.
    If there are short forms, then mention the full form in brackets. 
    they should be relevant for indian business news readers. no preamble postamble. no need for any headers. directly give upto 50 keywords mixed with nouns and keywords.
    I will use those keywords to filter news related to this topic. make sure that these keywords won't lead to a lot of false positives in terms of irrelevant content for users
    '''

In [9]:
model = 'gpt-4'

In [12]:
from openai import OpenAI

In [13]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=''
)

In [26]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [27]:
def _get_all_topics():
    topic_url = f'{BACKEND_URL}/public/articles/topics'
    r = requests.get(topic_url)
    all_topics = json.loads(r.text)
    # TODO: - add validations
    return all_topics['topics']

In [28]:
BACKEND_URL = 'https://yg4syp14u8.execute-api.ap-south-1.amazonaws.com/beta'

In [29]:
all_topics = _get_all_topics()

In [31]:
topic_mapping = {}

In [32]:
for topic in all_topics:
    topic_mapping[topic['topicId']] = topic['topicName']

In [34]:
topic_mapping[3] = 'Financial Independence, Retire Early (FIRE)'

In [51]:
def fetch_keywords_for_topic(topic_name, topic_id):
    res_gpt = client.chat.completions.create(model = model, messages = [{'role': 'system', 'content': ''}, {'role': 'user', 'content': get_prompt(topic_name)}])
    return topic_id, res_gpt.choices[0].message.content.strip()

In [37]:
sample = {1: topic_mapping[1], 2: topic_mapping[2]}

In [43]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [62]:
def get_gpt4_keywords_for_topics():
    start_time = time.time()
    responses = {}
    keywords = {}
    valid_responses = 0
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(fetch_keywords_for_topic, topic_name, topic_id) for topic_id, topic_name in topic_mapping.items()]
        
        for future in as_completed(futures):
            topic_id, keyword = future.result()
            if keyword:  # Timeout or other errors
                valid_responses += 1
                keywords[topic_id] = keyword
    print(f'done in {time.time() - start_time} seconds')
    return keywords

In [63]:
expansive_keywords = get_gpt4_keywords_for_topics()

done in 77.02781009674072 seconds


In [69]:
with PostgresDatabaseOperation() as cursor:
    sql = 'INSERT INTO topic_internal_representations (topic_id, description, approach_name) VALUES (%s, %s, %s)'
    for topic_id, keyword in expansive_keywords.items():
        cursor.execute(sql, (topic_id, keyword, 'gpt4-keywords-v2'))

In [61]:
k

{1: 'Savings Account, Budget Allocation, Expense Tracking, Financial Goal, Emergency Fund, Cash Flow, Asset Allocation, Investment Plan, Fixed Deposit, Mutual Funds, Recurring Deposit, Income Tax, Public Provident Fund (PPF), Life Insurance, Health Insurance, Retirement Plan, Employee Provident Fund (EPF), Financial Planning, Debt Reduction, Credit Card, Monthly Budget, Personal Finance, Household Expenses, Income Savings, Financial Literacy, Reserve Bank of India (RBI), State Bank of India (SBI), HDFC Bank, ICICI Bank, Securities and Exchange Board of India (SEBI), Pension Fund Regulatory and Development Authority (PFRDA), Life Insurance Corporation of India (LIC), National Savings Certificate (NSC), Tax Saving Bonds, National Pension System (NPS), Mahindra & Mahindra Financial Services, Bajaj Finserv, Reliance Capital, Kotak Mahindra Bank, Paytm Payment Bank, Axis Bank, IDFC First Bank, National Stock Exchange (NSE), Bombay Stock Exchange (BSE), IndusInd Bank, Union Budget.',
 2: 'De

In [57]:
k

{1: 'Financial Planning, Saving, Budgeting, Income, Expenses, Investments, Emergency Funds, Long-term saving, Short-term saving, Debt, Retirement, Fiscal discipline, Financial goals, Savings account, Wealth management, Monthly budget, Yearly budget, Taxes, Personal finance, Inflation, Financial literacy, Interest rates, Fixed deposit, Mutual funds, Economic cycle, Reserve Bank of India (RBI), Securities and Exchange Board of India (SEBI), Life Insurance Corporation (LIC), State Bank of India (SBI), HDFC Bank, ICICI Bank, Kotak Mahindra Bank, Axis Bank, YES Bank, Federal Bank, Canara Bank, Punjab National Bank, Mahindra Finance, Tata Capital, Aditya Birla Finance, Bajaj Finserv, Reliance Nippon Life Asset Management, UTI Asset Management Company, India Post Payments Bank, Indian Bank, Income Tax Department, Ministry of Finance, National Savings Certificate (NSC), Public Provident Fund (PPF), Employee Provident Fund (EPF).',
 2: 'Debt relief, Consolidation loan, Creditors, Insolvency, Ba

In [54]:
k

{1: 'Financial Planning, Savings Account, Monthly Budget, Household Expenses, Mutual Funds, Fixed Deposit, Saving Schemes, Public Provident Fund (PPF), Equity Linked Savings Scheme (ELSS), Debt, Income, Luxury Spending, Financial Goals, Retirement Funds, Investment, Essential Expenses, Cash Flow, Emergency Fund, Insurance, Taxes, Debt Repayment, Budget Surplus, Budget Deficit, Cost Cutting, Credit Card.\n\nReserve Bank of India (RBI), Securities and Exchange Board of India (SEBI), State Bank of India (SBI), Life Insurance Corporation of India (LIC), Employee Provident Fund Organisation (EPFO), Income Tax Department India, Infosys Ltd., Tata Consultancy Services (TCS), HDFC Bank, ICICI Bank, Axis Bank, Reliance Industries Ltd., Wipro, Mahindra & Mahindra, National Stock Exchange of India (NSE), Bombay Stock Exchange (BSE), National Pension System (NPS), Pradhan Mantri Jan Dhan Yojana (PMJDY), Union Budget of India, Central Board of Direct Taxes (CBDT).',
 2: "Debt Consolidation, Refinan